In [2]:
import csv
import itertools
import pandas as pd
import numpy as np
import networkx as nx

In [4]:
papers=[]
with open("train.txt", "r") as f:
    for x in f:
        token_list=x.rstrip("\n").split()
        papers.append(token_list)

graph=[]

for paper in papers:
    for pair in itertools.combinations(paper, 2):
        graph.append(pair)

graph_df=pd.DataFrame(graph, columns=['Author_A','Author_B'])
#Calculate weights
g_df = graph_df.groupby(['Author_A','Author_B']).size().reset_index().rename(columns={0:'count'})
#create undirectional graph 
ng = nx.convert_matrix.from_pandas_edgelist(g_df, source= 'Author_A', target = 'Author_B', edge_attr= 'count')
print(nx.info(ng))
print(g_df)
authorlist_A = g_df['Author_A'].to_list()
authorlist_B = g_df['Author_B'].to_list()
authorlist.extend(authorlist_B)
authorlist = list(set(authorlist)) # remove duplicates
authorlist_A = list(set(authorlist_A)) # remove duplicates
authorlist_B = list(set(authorlist_B)) # remove duplicates

Name: 
Type: Graph
Number of nodes: 3767
Number of edges: 16036
Average degree:   8.5139
      Author_A Author_B  count
0            0     1236     14
1            0     1655      9
2            0     1797      4
3            0     2414      1
4            0     2568      8
...        ...      ...    ...
16031      999     2038      1
16032      999     2250      4
16033      999     2600      1
16034      999     2905      3
16035      999     3926      1

[16036 rows x 3 columns]


NameError: name 'authorlist' is not defined

In [8]:
cnc = nx.common_neighbor_centrality(ng)

<generator object _apply_prediction.<locals>.<genexpr> at 0x000001FECE4DEF20>


In [9]:
[print (a,b,c) for a,b,c in cnc]

KeyError: '603'

In [ ]:
# this step takes ages Link prediction metrics are very slow to calculate
#cnc = nx.common_neighbor_centrality(ng)
feature_1 = []
for s,t in ng.edges():
    try:
        metric = nx.common_neighbor_centrality(ng, [(s,t)])
        for u,v,p in metric:
            feature_1.append(p)
    except:
        feature_1.append(0)

In [3]:
#getting all set of edges
r = csv.reader(open('no_headers_graph_pos.csv','r'))
edges = dict()
for edge in r:
    edges[(edge[0], edge[1])] = 1

In [174]:
%%time
###generating negative edges from given graph
import random

random.seed(1687)
missing_edges = set([])
while (len(missing_edges)<16036): # such that same number as positive edges
    a=random.choice(authorlist_A) # as the authorlist in the train set, contains all possible authors.
    b=random.choice(authorlist_B)
    tmp = edges.get((a,b),-1)
    if tmp == -1 and a!=b:
        try:
            if nx.shortest_path_length(ng,source=a,target=b) > 2 and (a,b) not in missing_edges and (b,a) not in missing_edges: 
                
                missing_edges.add((a,b))
            else:
                continue  
        except:
            if (a,b) not in missing_edges and (b,a) not in missing_edges:
                missing_edges.add((a,b))              
    else:
        continue

Wall time: 1.77 s


In [136]:
print(len(missing_edges))
df_neg = pd.DataFrame(list(missing_edges), columns=['Author_A', 'Author_B'])
df_neg = df_neg.groupby(['Author_A','Author_B']).size().reset_index().rename(columns={0:'count'})
df_neg['Author_A'] = df_neg['Author_A'].astype(str)
df_neg['Author_B'] = df_neg['Author_B'].astype(str)
ng_neg = nx.from_pandas_edgelist(df_neg, source= 'Author_A', target = 'Author_B', edge_attr= 'count')
print(df_neg)
df_pos = g_df
print(nx.info(ng_neg))
#sum(df_pos.duplicated())
y_pos = np.ones(len(df_pos))
y_neg = np.zeros(len(df_pos))

      Author_A Author_B  count
0            0     1087      1
1            0     1113      1
2            0     1780      1
3            0     1806      1
4            0     3245      1
...        ...      ...    ...
16031      998      533      1
16032      999     3391      1
16033      999      350      1
16034      999     3970      1
16035      999      507      1

[16036 rows x 3 columns]
Name: 
Type: Graph
Number of nodes: 3762
Number of edges: 16036
Average degree:   8.5253


In [6]:
feature_1_neg = []
for s,t in ng_neg.edges():
    try:
        metric = nx.common_neighbor_centrality(ng_neg, [(s,t)])
        for u,v,p in metric:
            feature_1_neg.append(p)
    except:
        feature_1_neg.append(0)

NameError: name 'ng_neg' is not defined

In [20]:
def feature_2(g):
    btwc = nx.edge_betweenness_centrality(g)
    feature_2 = []
    for s,t in g.edges():
        try:
            feature_2.append(btwc[(str(s),str(t))])
        except:
            feature_2.append(0) 
    return feature_2

In [21]:
def feature_3(g):
    feature_3 = []
    for s,t in g.edges():
        try:
            metric = nx.adamic_adar_index(g, [(s,t)])
            for u,v,p in metric:
                feature_3.append(p)
        except:
            feature_3.append(0)
    return feature_3

In [22]:
def feature_4(g):
    cc = nx.closeness_centrality(g)
    feature_4 = []
    for s,t in g.edges():
        try:
            feature_4.append(cc[str(s)])
        except:
            feature_4.append(0)
    return feature_4

In [23]:
def feature_5(g):
    dc = nx.degree_centrality(g)
    feature_5 = []
    for s,t in g.edges():
        try:
            feature_5.append(dc[str(s)])
        except:
            feature_5.append(0)
    return feature_5

In [ ]:
# need "community" information for that. Do we have that piece of information?
wic = nx.within_inter_cluster(ng)

In [24]:
def feature_6(g):
    feature_6 = []
    for s,t in g.edges():
        try:
            metric = nx.jaccard_coefficient(g, [(s,t)])
            for u,v,p in metric:
                feature_6.append(p)
        except:
            feature_6.append(0)
    return feature_6

In [25]:
def feature_7(g):
    feature_7 = []
    for s,t in g.edges():
        try:
            metric = nx.preferential_attachment(g, [(s,t)])
            for u,v,p in metric:
                feature_7.append(p)
        except:
            feature_7.append(0)
    return feature_7

In [26]:
def feature_8(g):
    pr = nx.pagerank(g)
    # pagerank Author_A.
    feature_8 = []
    for s,t in g.edges():
        try:
            rank = pr[s]
            feature_8.append(rank)
        except:
            feature_8.append(0)
    return feature_8

In [27]:
def feature_9(g):
    # pagerank Author_B.
    feature_9 = []
    for s,t in g.edges():
        try:
            rank = pr[t]
            feature_9.append(rank)
        except:
            feature_9.append(0)
    return feature_9

In [56]:
katz = nx.katz.katz_centrality(ng)
# Katz centrality Author_A - doesn't converge.
feature_10 = []
for s,t in ng.edges():
    try:
        kc = katz[s]
        feature_10.append(kc)
    except:
        feature_10.append(0)

PowerIterationFailedConvergence: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 1000 iterations')

In [ ]:
# Katz centrality Author_B
feature_11 = []
for s,t in ng.edges():
    try:
        kc = katz[t]
        feature_11.append(kc)
    except:
        feature_11.append(0)

In [ ]:
katz = nx.katz.katz_centrality(ng_neg)
# Katz centrality Author_A neg edges.
feature_10_neg = []
for s,t in ng_neg.edges():
    try:
        kc = katz[s]
        feature_10_neg.append(kc)
    except:
        feature_10_neg.append(0)

In [ ]:
# Katz centrality Author_B neg edges.
feature_11_neg = []
for s,t in ng_neg.edges():
    try:
        kc = katz[t]
        feature_11_neg.append(kc)
    except:
        feature_11_neg.append(0)

In [31]:
def feature_12_13(g):
    hits = nx.hits(g, max_iter=500)
    # for Author A.
    feature_12 = []
    feature_13 = []
    for s,t in g.edges():
        try:
            hit_i = hits[0][s]
            hit_j = hits[1][s]
            feature_12.append(hit_i)
            feature_13.append(hit_j)
        except:
            feature_12.append(0)
            feature_13.append(0)
    return feature_12, feature_13

In [29]:
def feature_14_15(g):
    # for Author B.
    feature_14 = []
    feature_15 = []
    for s,t in g.edges():
        try:
            hit_i = hits[0][t]
            hit_j = hits[1][t]
            feature_14.append(hit_i)
            feature_15.append(hit_j)
        except:
            feature_14.append(0)
            feature_15.append(0)
    return feature_14, feature_15

In [151]:
#df_pos['cnc'] = feature_1
df_pos['btwc'] = feature_2(ng)
df_pos['adar'] = feature_3(ng)
df_pos['cc'] = feature_4(ng)
df_pos['dc'] = feature_5(ng)
df_pos['jc'] = feature_6(ng)
df_pos['pa'] = feature_7(ng)
df_pos['pagerank_A'] = feature_8(ng)
df_pos['pagerank_B'] = feature_9(ng)
#df_pos['katz_A'] = feature_10
#df_pos['katz_B'] = feature_11
df_pos['hits_authority_A'], df_pos['hits_hubs_A'] = feature_12_13(ng)
df_pos['hits_authority_B'], df_pos['hits_hubs_B'] = feature_14_15(ng)
#df_neg['cnc'] = feature_1_neg
df_neg['btwc'] = feature_2(ng_neg)
df_neg['adar'] = feature_3(ng_neg)
df_neg['cc'] = feature_4(ng_neg)
df_neg['dc'] = feature_5(ng_neg)
df_neg['jc'] = feature_6(ng_neg)
df_neg['pa'] = feature_7(ng_neg)
df_neg['pagerank_A'] = feature_8(ng_neg)
df_neg['pagerank_B'] = feature_9(ng_neg)
#df_neg['katz_A'] = feature_10_neg
#df_neg['katz_B'] = feature_11_neg
df_neg['hits_authority_A'], df_neg['hits_hubs_A'] = feature_12_13(ng_neg)
df_neg['hits_authority_B'], df_neg['hits_hubs_B'] = feature_14_15(ng_neg)

In [152]:
import pickle
df_pos.to_csv("pos_training_data.csv")
df_neg.to_csv("neg_training_data.csv")
with open('pos_training_labels.ob', 'wb') as fp:
    pickle.dump(y_pos, fp)
with open('neg_training_labels.ob', 'wb') as fp:
    pickle.dump(y_neg, fp)
df_final_train = df_pos.append(df_neg,ignore_index=True)
y_final_train = np.concatenate((y_pos,y_neg))
df_final_train.to_csv("training_data_combined.csv")
with open('training_labels_combined.ob', 'wb') as fp:
    pickle.dump(y_final_train, fp)

In [153]:
df_final_train

,Author_A,Author_B,count,btwc,adar,cc,dc,jc,pa,pagerank_A,pagerank_B,hits_authority_A,hits_hubs_A,hits_authority_B,hits_hubs_B
0,0,1236,14,2.013822e-04,2.471649,0.187879,0.002124,0.428571,96,0.000220,0.000302,0.000003,0.000003,0.000057,0.000057
1,0,1655,9,6.067351e-05,2.976054,0.187879,0.002124,0.466667,112,0.000220,0.000376,0.000003,0.000003,0.000011,0.000011
2,0,1797,4,1.644754e-07,2.899858,0.187879,0.002124,0.700000,72,0.000220,0.000245,0.000003,0.000003,0.000003,0.000003
3,0,2414,1,4.159867e-06,2.471649,0.187879,0.002124,0.600000,64,0.000220,0.000223,0.000003,0.000003,0.000002,0.000002
4,0,2568,8,6.390372e-05,2.976054,0.187879,0.002124,0.466667,112,0.000220,0.000363,0.000003,0.000003,0.000018,0.000018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32067,998,533,1,2.600342e-04,0.000000,0.220148,0.000798,0.000000,30,0.000127,0.000313,0.000068,0.000068,0.000189,0.000189
32068,999,3391,1,1.108282e-04,0.000000,0.220148,0.000798,0.000000,12,0.000127,0.000162,0.000068,0.000068,0.000201,0.000201
32069,999,350,1,1.480084e-04,0.000000,0.250250,0.002127,0.000000,32,0.000253,0.000142,0.000248,0.000248,0.000099,0.000099
32070,999,3970,1,5.316321e-04,0.000000,0.245448,0.001861,0.000000,7,0.000256,0.000071,0.000216,0.000216,0.000000,0.000000


In [18]:
# public test data transformation.
test_df = pd.read_csv('test-public.csv')
test_df.drop(test_df.columns[[0]], axis=1, inplace=True)
test_df.columns=['Author_A','Author_B']
test_df = test_df.groupby(['Author_A','Author_B']).size().reset_index().rename(columns={0:'count'})
#create undirectional graph 
ng_test = nx.convert_matrix.from_pandas_edgelist(test_df, source= 'Author_A', target = 'Author_B', edge_attr= 'count')
ng_test

In [32]:

test_df['btwc'] = feature_2(ng_test)
test_df['adar'] = feature_3(ng_test)
test_df['cc'] = feature_4(ng_test)
test_df['dc'] = feature_5(ng_test)
test_df['jc'] = feature_6(ng_test)
test_df['pa'] = feature_7(ng_test)

test_df['pagerank_A'] = feature_8(ng_test)
test_df['pagerank_B'] = feature_9(ng_test)
#df_neg['katz_A'] = feature_10_neg
#df_neg['katz_B'] = feature_11_neg
test_df['hits_authority_A'], test_df['hits_hubs_A'] = feature_12_13(ng_test)
test_df['hits_authority_B'], test_df['hits_hubs_B'] = feature_14_15(ng_test)

In [34]:
test_df.to_csv("test_data.csv")

In [173]:
test_df

,Author_A,Author_B,count,btwc,adar,cc,dc,jc,pa,pagerank_A,pagerank_B,hits_authority_A,hits_hubs_A,hits_authority_B,hits_hubs_B
0,0,1236,14,2.013822e-04,2.471649,0.187879,0.002124,0.428571,96,0.000220,0.000561,3.166244e-06,3.166244e-06,0.000786,0.000786
1,0,1655,9,6.067351e-05,2.976054,0.187879,0.002124,0.466667,112,0.000220,0.000308,3.166244e-06,3.166244e-06,0.000272,0.000272
2,0,1797,4,1.644754e-07,2.899858,0.187879,0.002124,0.700000,72,0.000220,0.000195,3.166244e-06,3.166244e-06,0.000186,0.000186
3,0,2414,1,4.159867e-06,2.471649,0.187879,0.002124,0.600000,64,0.000220,0.000273,3.166244e-06,3.166244e-06,0.000310,0.000310
4,0,2568,8,6.390372e-05,2.976054,0.187879,0.002124,0.466667,112,0.000220,0.000474,3.166244e-06,3.166244e-06,0.000394,0.000394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16031,999,2038,1,4.753808e-04,0.000000,0.179502,0.000797,0.000000,3,0.000201,0.000225,3.083628e-06,3.083628e-06,0.000189,0.000189
16032,999,2250,4,4.753808e-04,0.000000,0.203988,0.001062,0.000000,4,0.000253,0.000196,8.021108e-06,8.021108e-06,0.000201,0.000201
16033,999,2600,1,4.753808e-04,0.000000,0.203988,0.001062,0.000000,4,0.000253,0.000152,8.021108e-06,8.021108e-06,0.000099,0.000099
16034,999,2905,3,4.753808e-04,0.000000,0.224377,0.001062,0.000000,4,0.000162,0.000000,1.353979e-04,1.353979e-04,0.000000,0.000000
